In [9]:
#блок получения информации о введённом пользователем фильме
import requests
from bs4 import BeautifulSoup
import telebot
import datetime
from telebot import types


with open("base.txt", "w", encoding="utf-8") as storage:
    storage.write(":::")

key = "k_ld3fatkm"
class Film:
    #ищем id фильма на сайтe imdb
    def __init__(self, film):
        try:
            r = requests.get(f"https://imdb-api.com/api/SearchMovie/{key}/{film}")
            data = r.json()
            self.id = data["results"][0]["id"]
        except Exception:
            return (None)

    #входит ли фильм в топ-250
    def top(self):
        r = requests.get(f"https://imdb-api.com/en/API/Top250Movies/{key}")
        data = r.json()
        try:
            for dicts in data["items"]:
                if dicts["id"] == self.id:
                    return "Да, фильм входит в топ!"
            return "Нет, фильм не входит в топ!"
        except Exception:
            return "К сожалению, не могу найти фильм."
    
    #рейтинг на imdb
    def rating(self):
        try: 
            r = requests.get(f"https://imdb-api.com/en/API/Ratings/{key}/{self.id}")
            data = r.json()
            return data["imDb"]
        except Exception:
            return "К сожалению, не удалось получить рейтинг."
    
    #трейлер
    def trailer(self):
        try:
            r = requests.get(f"https://imdb-api.com/en/API/Trailer/{key}/{self.id}")
            data = r.json()
            return data["link"]
        except Exception:
            return "К сожалению, не удалось найти трейлер."
    
    #описание с википедии
    def plot(self):
        try:
            r = requests.get(f"https://imdb-api.com/API/Wikipedia/{key}/{self.id}")
            data = r.json()
            return data["plotShort"]["plainText"]
        except Exception:
            return "К сожалению, информации не найдено."
    #постер
    def poster(self):
        try:
            r = requests.get(f"https://imdb-api.com/en/API/Posters/{key}/{self.id}")
            data = r.json()
            return data["posters"][0]["link"]
        except Exception:
            return "К сожалению, не удалось найти постер."

In [10]:
#отслеживание запроса
def user_add(user_name, user_state):
    with open("base.txt", "r", encoding="utf-8") as storage:
        text_storage = storage.read()
    with open("base.txt", "w", encoding="utf-8") as storage:
        storage.write(text_storage + user_name + "+++" + user_state + ":::")
def user_state_check(user_name):
    with open("base.txt", "r", encoding="utf-8") as storage:
        text_storage = storage.read()
        if text_storage.find(user_name) != -1:
            state = text_storage[(text_storage.find(user_name) + len(user_name) + 3) : (text_storage.find(user_name) + len(user_name) + 5)]
            return state
        return "ns"
def user_del(user_name, user_state):
    with open("base.txt", "r", encoding="utf-8") as storage:
        text_storage = storage.read()
        if  text_storage.find(user_name):
            text_storage = text_storage.replace(user_name + "+++" + user_state + ":::", "")
    with open("base.txt", "w", encoding="utf-8") as storage:
        storage.write(text_storage)

In [11]:
#блок получения информации по тв-программе
def programm(my_ch, user_name):
    url = 'http://www.vsetv.com/schedule.html'
    response = requests.get(url)
    
    tree = BeautifulSoup(response.content, 'html.parser')
    
    channel = tree.find_all('div', {'class' : "chname"})
    chs = list()
    for i in range(len(channel)):
        chs.append(channel[i].find('td', {'class': "channeltitle"}).text)
    if my_ch in chs:
        n = chs.index(my_ch)
        user_del(user_name, "tv")
        return (f"Отлично, у меня есть информация о канале {my_ch}!\n"
              f"Вот ссылка на программу передач канала {my_ch}:  http://www.vsetv.com/{channel[n].a.get('href')}.")
    else:
        return ("Извините, не могу помочь. Попробуйте ввести другой канал.")

In [12]:
#оформление тг-бота
bot = telebot.TeleBot("5947323758:AAEaCjiKvxa-0yqd47NOFdUAi-lSiSu4jmE")

@bot.message_handler(commands = ["start"])
def start(message):
    if user_state_check(str(message.chat.id)) == "tv":
        user_del(str(message.chat.id), "tv")
    markup = types.InlineKeyboardMarkup()
    tv_b = types.InlineKeyboardButton(text = "Программу передач!", callback_data = "tv")
    film_b = types.InlineKeyboardButton(text = "Информацию о фильме!", callback_data = "film")
    markup.add(tv_b, film_b)
    bot.send_message(message.chat.id, f"{message.from_user.first_name}, вы хотите узнать телепрограмму на {datetime.date.fromtimestamp(message.date)} или найти информацию о каком-либо фильме?", reply_markup = markup)
@bot.callback_query_handler(func = lambda call: True)
def reply(call):
    if call.data == "tv":
        bot.send_message(call.message.chat.id, "Введите, пожалуйста, команду /tv")
    elif call.data == "film":
        bot.send_message(call.message.chat.id, "Введите, пожалуйста, команду /film")
@bot.message_handler(commands=["tv"])
def tv_reply(message):
    user_add(str(message.chat.id), "tv")
    bot.send_message(message.chat.id, "Введите, пожалуйста, название канала на русском языке с заглавной буквы.")

@bot.message_handler(commands=["film"])
def film_reply(message):
    if user_state_check(str(message.chat.id)) == "tv":
        user_del(str(message.chat.id), "tv")
    user_add(str(message.chat.id), "fm")
    bot.send_message(message.chat.id, "Введите, пожалуйста, название фильма, который вас интересует, на английском языке.")




In [13]:
@bot.message_handler(content_types=["text"])
def reply_result(message):
    state = user_state_check(str(message.chat.id))
    if state == "tv":
        bot.send_message(message.chat.id, f"{programm(message.text, str(message.chat.id))}")
    if state == "fm":
        bot.send_message(message.chat.id, f"Входит ли '{message.text}' в топ250 IMDb?\n"
                         f"{Film(message.text).top()}\n"
                         "\n"
                         "Какой рейтинг у этого фильма?\n"
                         f"{Film(message.text).rating()}\n"
                         "\n"
                         "Есть ли ссылка на трейлер и постер?\n"
                         f"{Film(message.text).trailer()}\n"
                         f"{Film(message.text).poster()}\n"
                         "\n"
                         f"Краткое описание '{message.text}'?\n"
                         f"{Film(message.text).plot()}\n")
        user_del(str(message.chat.id), "fm")
    if state == "ns":
        markup = types.InlineKeyboardMarkup()
        tv_b = types.InlineKeyboardButton(text = "Программу передач!", callback_data = "tv")
        film_b = types.InlineKeyboardButton(text = "Информацию о фильме!", callback_data = "film")
        markup.add(tv_b, film_b)
        bot.send_message(message.chat.id, f"{message.from_user.first_name}, вы хотите узнать телепрограмму на {datetime.date.fromtimestamp(message.date)} или найти информацию о каком-либо фильме?",
                         reply_markup=markup)

                                             
bot.polling(none_stop = True)